In [ ]:
# !pip install -q xlstm ninja

In [5]:
# !git clone -q https://github.com/NX-AI/xlstm.git
# !git clone -q https://github.com/IBM/CrossViT.git
# !cd xlstm && pip install -e .
# content = """torch
# torchvision
# timm==0.4.12
# fvcore
# einops
# submitit"""
# with open('CrossViT/requirements.txt', 'w') as f:
#     f.write(content)
# !cd CrossViT && pip install -q -r requirements.txt && pip install -q ninja
# !cp -r xlstm CrossViT/models
# !cp -r /content/crossvit.py /content/CrossViT/models/crossvit.py
# !wget -q https://wallpaperaccess.com/full/4723250.jpg -O test.jpg

In [6]:
import os
import shutil

base_dir = "CrossViT/data"
os.makedirs(base_dir, exist_ok=True)

train_data = {
    'class1': ['img1.jpeg', 'img2.jpeg', 'img3.jpeg', 'img4.jpeg', 'img5.jpeg', 'img6.jpeg','img7.jpeg', 'img9.jpeg', 'img10.jpeg'],
    'class2': ['img1.jpeg', 'img2.jpeg', 'img3.jpeg', 'img4.jpeg', 'img5.jpeg', 'img6.jpeg','img7.jpeg', 'img9.jpeg', 'img10.jpeg']
}

val_data = {
    'class1': ['img3.jpeg', 'img2.jpeg', 'img3.jpeg', 'img4.jpeg', 'img5.jpeg', 'img6.jpeg','img7.jpeg', 'img9.jpeg', 'img10.jpeg'],
    'class2': ['img4.jpeg', 'img2.jpeg', 'img3.jpeg', 'img4.jpeg', 'img5.jpeg', 'img6.jpeg','img7.jpeg', 'img9.jpeg', 'img10.jpeg']
}
image_url = "https://wallpaperaccess.com/full/4723250.jpg"

def fn(name, data):
    train_dir = os.path.join(base_dir, name)
    os.makedirs(train_dir, exist_ok=True)

    for class_name, images in data.items():
        class_dir = os.path.join(train_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        for img in images:
            img_path = os.path.join(class_dir, img)
            shutil.copy('test.jpg', img_path)

fn('train', train_data)
fn('val', val_data)

In [7]:
from CrossViT.models.crossvit import VisionTransformer

/root/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
vit = VisionTransformer()
# vit

In [9]:
import torch
from CrossViT.models.crossvit import VisionTransformer
vit = VisionTransformer().to('cuda')

In [10]:
IMG_SIZE = 256
input = torch.randn(8, 3, IMG_SIZE, IMG_SIZE).to("cuda")
output = vit(input)
output.shape

torch.Size([8, 1000])

In [ ]:
!python /content/CrossViT/main.py --model crossvit_9_dagger_224 --batch-size 8 --data-path "/content/CrossViT/data"

In [ ]:
import torch

from xlstm.xlstm import (
    xLSTMBlockStack,
    xLSTMBlockStackConfig,
    mLSTMBlockConfig,
    mLSTMLayerConfig,
    sLSTMBlockConfig,
    sLSTMLayerConfig,
    FeedForwardConfig,
)

In [ ]:
cfg = xLSTMBlockStackConfig(
    mlstm_block=mLSTMBlockConfig(
        mlstm=mLSTMLayerConfig(
            conv1d_kernel_size=4, qkv_proj_blocksize=4, num_heads=4
        )
    ),
    slstm_block=sLSTMBlockConfig(
        slstm=sLSTMLayerConfig(
            backend="cuda",
            num_heads=4,
            conv1d_kernel_size=4,
            bias_init="powerlaw_blockdependent",
        ),
        feedforward=FeedForwardConfig(proj_factor=1.3, act_fn="gelu"),
    ),
    context_length=256,
    num_blocks=7,
    embedding_dim=128,
    slstm_at=[1],

)

xlstm_stack = xLSTMBlockStack(cfg)
xlstm_stack

In [ ]:
cfg = xLSTMBlockStackConfig(
    mlstm_block=mLSTMBlockConfig(
        mlstm=mLSTMLayerConfig(
            conv1d_kernel_size=4, qkv_proj_blocksize=4, num_heads=4
        )
    ),
    slstm_block=None,
    context_length=256,
    num_blocks=1,
    embedding_dim=128,
)

xlstm_stack = xLSTMBlockStack(cfg)
# xlstm_stack
x = torch.randn(4, 256, 128).to("cuda")
xlstm_stack = xlstm_stack.to("cuda")
y = xlstm_stack(x)
y.shape

torch.Size([4, 256, 128])

In [ ]:
cfg = xLSTMBlockStackConfig(
    mlstm_block=None,
    slstm_block=sLSTMBlockConfig(
        slstm=sLSTMLayerConfig(
            backend="cuda",
            num_heads=4,
            conv1d_kernel_size=4,
            bias_init="powerlaw_blockdependent",
        ),
        feedforward=FeedForwardConfig(proj_factor=1.3, act_fn="gelu"),
    ),
    context_length=256,
    num_blocks=1,
    embedding_dim=128,
)

xlstm_stack = xLSTMBlockStack(cfg)
# xlstm_stack

In [ ]:
x = torch.randn(1, 4, 256, 128).to("cuda")
xlstm_stack = xlstm_stack.to("cuda")
y = xlstm_stack(x)
y.shape